In [38]:
from bbq.analyse import *
import pandas as pd
from datetime import datetime, timedelta
from talib.abstract import *
import talib
import mplfinance as mpf
from bbq.data.stockdb import StockDB
from bbq.data.funddb import FundDB
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import json

db_stock = StockDB()
db_fund = FundDB()
db_stock.init()
db_fund.init()

data = await db_stock.load_stock_daily(filter={'code': 'sz002132'}, limit=60, sort=[('trade_date', -1)])
data = data[::-1]

name = await db_stock.load_stock_info(filter={'code': 'sz002132'}, projection=['name'])
data['name'] = name.iloc[0]['name']


data_fund = await db_fund.load_fund_daily(filter={'code': '159949'}, limit=60, sort=[('trade_date', -1)])
data_fund = data_fund[::-1]

name_fund = await db_fund.load_fund_info(filter={'code': '159949'}, projection=['short_name'])
data_fund['name'] = name_fund.iloc[0]['short_name']

2020-12-26 16:37:48,703 - StockDB - DEBUG - 加载股票日线, kwargs={'filter': {'code': 'sz002132'}, 'limit': 60, 'sort': [('trade_date', -1)]}
2020-12-26 16:37:48,764 - StockDB - DEBUG - 加载日线数据成功 size=60
2020-12-26 16:37:48,766 - StockDB - DEBUG - 加载股票信息, kwargs={'filter': {'code': 'sz002132'}, 'projection': ['name']} ...
2020-12-26 16:37:48,802 - StockDB - DEBUG - 加载股票信息成功 size=1
2020-12-26 16:37:48,804 - FundDB - DEBUG - 加载场内基金日线数据, kwargs={'filter': {'code': '159949'}, 'limit': 60, 'sort': [('trade_date', -1)]}
2020-12-26 16:37:49,066 - FundDB - DEBUG - 加载场内基金日线数据,成功, size=60
2020-12-26 16:37:49,068 - FundDB - DEBUG - 加载基金列表, kwargs={'filter': {'code': '159949'}, 'projection': ['short_name']}
2020-12-26 16:37:49,101 - FundDB - DEBUG - 加载基金列表成功, size=1


In [42]:
data = data_fund

k_data = list(zip(data['open'], data['close'], data['low'], data['high']))
trade_date = [d.strftime('%Y-%m-%d') for d in data['trade_date']]


def kline_tooltip_fmt_func():
    return JsCode("""function(obj){
                function tips_str(pre, now, tag, field, unit) {
                    var tips = tag + ':&nbsp;&nbsp;&nbsp;&nbsp;';
                    var span = '';
                    
                    if (field == 'open' || field == 'close' ||
                        field == 'low' || field == 'high') {
                        var fixed = 3;
                        if (now['code'].startsWith('s')) {
                            fixed = 2;
                        }
                        if (pre != null) {
                            var chg = (now[field] - pre['close']).toFixed(fixed);
                            var rate = (chg * 100 / pre['close']).toFixed(2);

                            if (rate >= 0) {
                                span = '<span style="color: #F11300;">' + now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + chg + ')</span><br>';
                            } else {
                                span = '<span style="color: #00A800;">' + now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + chg + ')</span><br>';
                            }

                        } else {
                            span = now[field].toFixed(fixed) + '&nbsp;&nbsp;(' + rate + '%,&nbsp;' + rate + ')<br>';
                        }
                    } else {
                        if (field == 'volume') {
                            span = (now[field] / 1000000.0).toFixed(2) + '&nbsp;万手<br>';
                        }
                        if (field == 'turn_over') {
                            span = now[field] + '%<br>'
                        }
                    }
                    return tips + span;

                }
                
                var pre_data = null;
                var now_data = kdata[trade_date[obj.dataIndex]];
                if (obj.dataIndex > 0) {
                    pre_data = kdata[trade_date[obj.dataIndex-1]];
                }
                var title = now_data['code'] + '&nbsp;&nbsp;' + trade_date[obj.dataIndex] + '<br><br>';

                if ('name' in now_data) {
                    title = now_data['name'] + '(' + now_data['code'] + ')&nbsp;&nbsp;' + trade_date[obj.dataIndex] + '</div><br>';
                }
                
                return title + 
                    tips_str(pre_data, now_data, '开盘', 'open') + 
                    tips_str(pre_data, now_data, '最高', 'high') + 
                    tips_str(pre_data, now_data, '最低', 'low') + 
                    tips_str(pre_data, now_data, '收盘', 'close') + 
                    tips_str(pre_data, now_data, '成交', 'volume') + 
                    tips_str(pre_data, now_data, '换手', 'turn_over');
                 }""")

def kline_orig_data(data):
    origData = data[:]
    origData['trade_date'] = origData['trade_date'].apply(lambda d: d.strftime('%Y-%m-%d'))
    origData.index = origData['trade_date']
    origData.fillna('-', inplace=True)
    return origData.to_dict('index')

kline = Kline()
kline.add_js_funcs('var kdata={}'.format(kline_orig_data(data)))
kline.add_js_funcs('var trade_date={}'.format(trade_date))
kline.add_xaxis(trade_date)
kline.add_yaxis('日线', k_data,
                itemstyle_opts=opts.ItemStyleOpts(
                    color='#F11300',
                    color0='#00A800',
                ),
#                markline_opts=opts.MarkLineOpts(
#                    data=[opts.MarkLineItem(type_="max", value_dim="highest")])
                tooltip_opts=opts.TooltipOpts(
                    formatter=kline_tooltip_fmt_func())
               )
kline.set_global_opts(xaxis_opts=opts.AxisOpts(is_scale=True),
                     yaxis_opts=opts.AxisOpts(
                         is_scale=True,
                         splitarea_opts=opts.SplitAreaOpts(
                             is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                         ),
                     ),
                      datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%", filter_mode='none')],
                      title_opts=opts.TitleOpts(title="Kline-MarkLine"))

ma5 = MA(data['close'], timeperiod=5)
ma5 = [round(v, 3) for v in ma5]

ma10 = MA(data['close'], timeperiod=10)
ma10 = [round(v, 3) for v in ma10]

line = Line()
line.add_xaxis(trade_date)
line.add_yaxis('ma5', ma5, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('ma10', ma10, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')

#scatter = Scatter()
#scatter.add_xaxis(trade_date)
#scatter.add_yaxis('买入', [data.iloc[30]['high']], )
#line.render_notebook()

kline.overlap(line)
kline.render_notebook()
#

In [44]:
data


,volume,open,high,low,close,turnover,trade_date,code,name
59,652345436,1.027,1.032,1.012,1.014,0.0,2020-09-24,159949,华安创业板50ETF
58,511111481,1.018,1.029,1.010,1.017,0.0,2020-09-25,159949,华安创业板50ETF
57,471972302,1.020,1.024,1.008,1.010,0.0,2020-09-28,159949,华安创业板50ETF
56,630676114,1.016,1.032,1.009,1.025,0.0,2020-09-29,159949,华安创业板50ETF
55,708717401,1.030,1.045,1.026,1.033,0.0,2020-09-30,159949,华安创业板50ETF
54,810093710,1.057,1.082,1.055,1.080,0.0,2020-10-09,159949,华安创业板50ETF
53,1190138252,1.090,1.122,1.088,1.120,0.0,2020-10-12,159949,华安创业板50ETF
52,916744497,1.118,1.128,1.110,1.122,0.0,2020-10-13,159949,华安创业板50ETF
51,672171483,1.120,1.128,1.113,1.117,0.0,2020-10-14,159949,华安创业板50ETF
50,611618763,1.119,1.122,1.107,1.110,0.0,2020-10-15,159949,华安创业板50ETF
